### Jordan Bramble

### Homework 5

#### 6.8 problem 2

##### b.

iii. is true.  less flexible, so as variance decreases, and bias increases less. we get better prediction accuracy

#### 6.8 problem 3

##### a.

iv. training RSS will decrease to OLS values

#### b.

ii. increasing s will give us better "feature selection", up to an optimal point, and then we will start minimizing important features and losing "information"

#### c. 

iii. when s = 0 there should be no variance. as s increases, more betas are becoming non-zero and thus variance should increase

#### 6.8 problem 9